# Sequence Level Features and Analysis

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from scipy.stats import ttest_1samp, poisson
from Bio import pairwise2
from Bio.pairwise2 import format_alignment



pd.set_option('display.float_format', '{:.10e}'.format)
pd.set_option('display.max_colwidth', 30)

In [2]:
csv_file = "../Data/R12-clean.csv"
df = pd.read_csv(csv_file)
df.head(5)

,Sequence,Copy Num,Length
0,AGTGCCATCGTGCGTATCCTTCACTC...,91,98
1,AGTGCCATCGTGCGTATCCTTCACGT...,86,98
2,AGTGCCATCGTGCGTATCCTGAACAT...,83,98
3,AGTGCCATCGTGCGTATCCCGCTCCG...,80,98
4,AGTGCCATCGTGCGTATCCTGAACAT...,78,98


### 1. k-mer Extraction

In [3]:
def extract_kmers(seq, k):
    """Extracts kmers from a sequence. Saves position and kmer chunk to preserve order and distance information for downstream secondary structure analysis. Collects kmers from all k reading frames."""
    all_kmers = [(i, seq[i:i+k]) for i in range(len(seq)-k+1)]
    return all_kmers

In [5]:
def getkmers(data, k):
    """Performs row-level extraction of kmers and returns long-form df."""
    kmer_data = []
    for idx, row in df.iterrows():
        kmers = extract_kmers(row['Sequence'], k)
        for position, kmer in kmers:
            kmer_data.append({'Sequence_ID': idx, 'Position': position, 'k-mer': kmer})
    
    kmer_df = pd.DataFrame(kmer_data)
    return kmer_df

In [6]:
getkmers(data=df, k=10).head()

,Sequence_ID,Position,k-mer
0,0,0,AGTGCCATCG
1,0,1,GTGCCATCGT
2,0,2,TGCCATCGTG
3,0,3,GCCATCGTGC
4,0,4,CCATCGTGCG


### 2. Search for motifs in variable regions.  
We previously considered a naive kmer search but there were too many false positives because high frequency kmers were arising from CRs. We pursue a new approach where we look for motifs only in the VRs and later add back potential overhangs into the CRs when we apply downstream secondary structure analysis to confirm active motifs.  

#### Designed constant regions of selection library: 
The following describes the template of the starting selection library:

forward primer | AAGTGCCATCGTGCGTATCC | 20 bp  
variable region 1 | (N)^22 | 22 bp  
mipomersen loading | GCGAAGCAGACTGAGGC | 17 bp  
variable region 2 | (N)^21 | 21 bp  
reverse primer | GTAGACTGGAGACACGACGA | 20 bp  

Boundary positions are approximate because of PCR mutations/sequencing errors.  For sequences of length 98, the orignal library size (nb: 100bp but NGS discards 1st and last reads), we estimate the boundary positions but allow a tuneable buffer of a few bps to account for the errors.  For some sequences of non-standard length, we perform pairwise alignment with CRs to more accurately determine the boundary positions.

In [97]:
# Define alignment function
def align(cr, seq):
    # Perform local alignment
    alignments = pairwise2.align.localms(seq, cr, 2, -1, -2, -0.5)
    best_alignment = alignments[0]
    
    # Extract aligned sequences
    aligned_seq1 = best_alignment.seqA
    aligned_seq2 = best_alignment.seqB

    # Map the aligned region back to PADNA_1
    start_in_seq2 = best_alignment.start + 1  # 1-based index for start
    end_in_seq2 = best_alignment.start + aligned_seq2.replace("-", "").rfind(next(c for c in reversed(aligned_seq2) if c != '-')) + 1

    # Display alignment and summary
    print(format_alignment(*best_alignment))
    print(f"Start = {start_in_seq2}, End = {end_in_seq2}")

# Define sequences
PADNA_1 = "AGTGCCATCGTGCGTATCCTTCACTCCTTGCTCGACAAGAAGCGAAGCAGACTGAGGCGTCCGATGGTCTAATTCTTCAGTAGACTGGAGACACGACG"
FP = "AAGTGCCATCGTGCGTATCC"

# Run alignment
align(FP, PADNA_1)

1 AGTGCCATCGTGCGTATCC
  |||||||||||||||||||
2 AGTGCCATCGTGCGTATCC
  Score=38

Start = 2, End = 21


In [106]:
# Look at one 98 bp sequence, PADNA-1
PADNA_1 = "AGTGCCATCGTGCGTATCCTTCACTCCTTGCTCGACAAGAAGCGAAGCAGACTGAGGCGTCCGATGGTCTAATTCTTCA GTAGACTGGAGACACGACG"
FP = "AAGTGCCATCGTGCGTATCC"
MIP = "GCGAAGCAGACTGAGGC"
RP = "GTAGACTGGAGACACGACGA"

align(MIP, PADNA_5)

42 GCGAAGCAGACTGAGGC
   |||||||||||||||||
 1 GCGAAGCAGACTGAGGC
  Score=34

Start = 42, End = 58


In [102]:
PADNA_5 = df.iloc[4,0]
print(PADNA_5)

AGTGCCATCGTGCGTATCCTGAACATAGACGTTTAGTCTATGCGAAGCAGACTGAGGCGCGTGGATAGCCTATTTTCGGGTAGACTGGAGACACGACG


In [103]:
align(RP, PADNA_5)

80 GTAGACTGGAGACACGACG
   |||||||||||||||||||
 1 GTAGACTGGAGACACGACG
  Score=38

Start = 80, End = 99


In [104]:
align(FP, PADNA_5)

1 AGTGCCATCGTGCGTATCC
  |||||||||||||||||||
2 AGTGCCATCGTGCGTATCC
  Score=38

Start = 2, End = 21
